## Creating a Language Model from scratch completely Locally

In [30]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu' #using cpu due to apple silicon restrictions
print(device)
block_size = 8
batch_size = 4
max_iters = 10000
learning_rate = 3e-4

cpu


In [3]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '#', '$', '%', '&', '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”', '•', '™', '\ufeff']


#### Setting encoders and decoders and using tensors to store data

In [4]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)

In [6]:
# example
print(encode('hello'))
print(decode(encode('hello')))

[62, 59, 66, 66, 69]
hello


#### Splitting data into training and validation sets

In [10]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

# function to get inputs and targets


def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)  # for cuda(not available on mac)
    return x, y


x, y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)

tensor([158779,  72367,  58131, 173629])
inputs:
tensor([[85,  1, 73, 55, 63, 58,  1, 74],
        [72, 66, 59, 74,  1, 70, 69, 70],
        [68, 61, 66, 79, 12,  0,  0, 84],
        [77, 69, 72, 66, 58, 12, 85,  0]])
targets:
tensor([[ 1, 73, 55, 63, 58,  1, 74, 62],
        [66, 59, 74,  1, 70, 69, 70, 70],
        [61, 66, 79, 12,  0,  0, 84, 35],
        [69, 72, 66, 58, 12, 85,  0,  0]])


#### Creating the Bigram Language Model

In [28]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) # nn.Embedding is a learnable parameter
    
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss  # Return both logits and loss

    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, _ = self.forward(index)  # Unpack logits and ignore loss
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) #(B, T+1)
        
        return index

In [29]:
# Example usage:

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_indices = m.generate(context, max_new_tokens=500)[0].tolist()

generated_chars = decode(generated_indices)
print(generated_chars)

AVTKGl¤~pQg¡A[wl\Ki¡\A¤DUld¢vNN\aQGHrsAg£Eai¢fKdRPqVebY{x£¡~MNnXmoM¢xs}tUaXnE\c|¤nrZKLYteF]R vjNJ|{~WZ{eG} wGujE\K¤N~BUFT£Y`mMd^wYzgcce|¤L]oObnU{yokdSf|KnYnxqAeDMe~kAWGX\_]hEEa`QgMrnUnJQzgNh¤W|Wz£T\Fe£\]BoDfRnloGMQ}Vi¢[ LXgbX[_iSxf[LpvhDsF`c¡kb{W£i¤[szMDZDy¡¡HqJJom|tyy]E[TKjtPVnQd`ANHSTC|tfiGaYlDrn¤d\dkibNOcGQANGG\FxUa~|BmwwfYXm^l


#### Training Loop

In [31]:
# create a pytorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True) # previous data doesnt affect the current data
    loss.backward()
    optimizer.step()
print(loss.item())

tensor([139470, 141111,  28588, 158862])
tensor([139213, 109677,  82366, 168850])
tensor([121891, 116952, 145543, 175621])
tensor([ 81485,  40414,  12702, 173369])
tensor([155015,   6967, 102491, 114914])
tensor([ 18114, 142413, 107374,  74513])
tensor([157191, 147805,  77208,  38181])
tensor([50563, 89739, 87258, 88385])
tensor([107954, 149926, 124717, 142404])
tensor([ 81094,   3776, 144395, 121238])
tensor([139860, 156563, 159268,  30622])
tensor([95663, 36433, 98814, 55930])
tensor([ 82955,  53135, 171949,  43458])
tensor([143048,  79684,  82806,   1974])
tensor([151355, 119258, 105677, 179413])
tensor([171462, 111032,  50920, 105893])
tensor([147604,  53576,  36313, 168988])
tensor([170503, 141440,  61680,   3528])
tensor([ 36632, 169870, 131168,  12731])
tensor([161146, 136465,  85186,  46733])
tensor([165805,  60204,  85686, 139982])
tensor([142818, 109596,  10844,   9202])
tensor([180695,  83343,  80553,  80330])
tensor([ 50799, 155124,  10685, 159928])
tensor([  1731,  30190, 